In [260]:
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor

url = "https://api.hypixel.net/skyblock/auctions"

r = requests.get(url).json()
total_pages = int(r['totalPages'])
total_pages
df3 = pd.DataFrame()
def fetch_auction_house(page_number:int) -> pd.DataFrame:
        response = requests.get(f'{url}?page={page_number}')
        auctions_data = response.json()['auctions']
        df = pd.DataFrame(auctions_data)
        df.drop(columns=['auctioneer','profile_id','coop', 'extra','item_lore','tier','item_bytes','claimed_bidders','item_uuid','bids'],inplace=True)
        return df

def clean_dataframe(df:pd.DataFrame) -> pd.DataFrame:
        df['start'] = pd.to_datetime(df['start'], unit='ms')
        df['end'] = pd.to_datetime(df['end'], unit='ms')
        df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
        return df



for i in range(total_pages):
   df3 = pd.concat( [df3, fetch_auction_house(i)])

df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,df4c2f41d16d472aaa6897b4a15152f8,2024-07-14 04:02:31.898,2024-07-14 10:02:31.898,Training Dummy,misc,294000,False,0,2024-07-14 04:02:31.898,True
1,dc95516691e04c5f84038da295c142a6,2024-07-14 04:02:31.666,2024-07-14 10:02:31.666,Experience Artifact,accessories,910000,False,0,2024-07-14 04:02:31.666,True
2,ac3d277a985b40d6b4050c9cb9e9de79,2024-07-14 04:02:31.588,2024-07-16 04:02:31.588,[Lvl 52] Mooshroom Cow,misc,8500000,False,0,2024-07-14 04:02:31.588,True
3,3d01f47184614567a7416203a2c3c5b5,2024-07-14 04:02:31.519,2024-07-16 04:02:31.519,Jaded Sorrow Boots,armor,13700000,False,0,2024-07-14 04:02:31.519,True
4,f83dca74698e45c7b8e49a17aefd5406,2024-07-14 04:02:31.300,2024-07-14 10:02:31.300,Velvet Top Hat,armor,84999,False,0,2024-07-14 04:02:31.300,True
...,...,...,...,...,...,...,...,...,...,...
574,16da74bf47f14517baa303c43d7bb757,2024-06-30 03:03:07.235,2024-07-14 05:03:07.235,Judgement Core,misc,406000000,False,0,2024-06-30 03:03:07.235,True
575,b554ea6158f1409fb742e6fd51c5eaa2,2024-06-30 02:38:49.438,2024-07-14 04:38:49.438,Plushie Reindeer Skin,misc,499999999,False,0,2024-06-30 02:38:49.438,True
576,4755bb995d68471d83f9de70d6a6f58d,2024-06-30 02:25:28.835,2024-07-14 04:25:28.835,Crochet Tiger Plushie,misc,26000000,False,0,2024-06-30 02:25:28.835,True
577,a7b5a37a604e47cc933d110a81c12c16,2024-06-30 02:25:06.798,2024-07-14 04:25:06.798,Barry Century Cake,misc,50000000,False,0,2024-06-30 02:25:06.798,True


In [288]:
# Using threading improved avg runtime from 25 - 30seconds to 3 - 4 (Thanks ChatGPT for the help)

url = "https://api.hypixel.net/skyblock/auctions"
r = requests.get(url).json()
total_pages = int(r['totalPages'])


# For loops through the total_number of pages on the market place, requesting JSON for each page
# Each page contains a max of 1000 Items
def fetch_auction_house(page_number: int) -> pd.DataFrame:
    response = requests.get(f'{url}?page={page_number}')
    auctions_data = response.json()['auctions']
    df = pd.DataFrame(auctions_data)
    return df
# Clean DataFrame
# Remove unnescessary code
# Change all timestamps from UNIX to yyyy/mm/dd hour/min/sec/ms format
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df.drop(columns=['auctioneer', 'profile_id', 'coop', 'extra', 'item_lore', 'tier', 'item_bytes', 'claimed_bidders', 'item_uuid', 'bids'], inplace=True)
    df['start'] = pd.to_datetime(df['start'], unit='ms')
    df['end'] = pd.to_datetime(df['end'], unit='ms')
    df['last_updated'] = pd.to_datetime(df['last_updated'], unit='ms')
    return df


#ChatGPT helped me with this one, ill need someone to explain this to me
# Use ThreadPoolExecutor to fetch pages concurrently
def fetch_all_pages_concurrently(total_pages: int) -> pd.DataFrame:
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(fetch_auction_house, i) for i in range(total_pages)]
        result = pd.concat([future.result() for future in futures], ignore_index=True)
    return result

df3 = fetch_all_pages_concurrently(total_pages)
df = clean_dataframe(df3)

df

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,8bd3d8e4a846432f99ccee9f23c2d78e,2024-07-14 05:38:32.181,2024-07-16 05:38:32.181,Purple Gift Talisman,accessories,3100000,False,0,2024-07-14 05:38:32.181,True
1,9c887d5c3f87443d87d5bdf6cd4e9de4,2024-07-14 05:38:31.843,2024-07-16 05:38:31.843,[Lvl 1] Rat,misc,199999,False,0,2024-07-14 05:38:31.843,True
2,34f580ab8c3f4ac28fbebe8f2f06cef4,2024-07-14 05:38:31.646,2024-07-16 05:38:31.646,Toucan Parrot Skin,misc,50000000,False,0,2024-07-14 05:38:31.646,True
3,098627e7fa354957b8325a897f7cbf2a,2024-07-14 05:38:31.534,2024-07-14 08:38:31.534,Sharp Pooch Sword,weapon,500,False,0,2024-07-14 05:38:31.534,False
4,372b181826f94fdfb7bc6129345f15e6,2024-07-14 05:38:31.465,2024-07-14 11:38:31.465,Earthworm Ensemble Vinyl,misc,10000,False,0,2024-07-14 05:38:31.465,True
...,...,...,...,...,...,...,...,...,...,...
58176,98dc0f6e073d4ca6b2338839d510e71d,2024-06-30 03:58:54.087,2024-07-14 05:58:54.087,Molten Cloak,misc,14500000,False,0,2024-06-30 03:58:54.087,True
58177,2de2dfedd2d74275b1d8e38fba73a278,2024-06-30 03:57:43.065,2024-07-14 05:57:43.065,Molten Cloak,misc,10000000,False,0,2024-06-30 03:57:43.065,True
58178,f6537a017a824d949a3efb5a5ba12ff7,2024-06-30 03:56:31.170,2024-07-14 05:56:31.170,Molten Belt,misc,2000000,False,0,2024-06-30 03:56:31.170,True
58179,1c69d16eb8434598b2f1bc3a5dcf1e77,2024-06-30 03:54:00.997,2024-07-14 05:54:00.997,Castle Barn Skin,blocks,128888888,False,0,2024-06-30 03:54:00.997,True


In [266]:
def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return "Item not with specified Parameters"
    return filtered_df

find_item('Portal Dye', 79000000)

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
355,/viewauction 852a02bdb21e47838bdce94c1e7b5fcf,2024-07-14 04:01:38.854,2024-07-14 10:01:38.854,Portal Dye,misc,76500000,False,0,2024-07-14 04:01:38.854,True
385,/viewauction f9eb03ce41744901bd0b4856513dc268,2024-07-14 04:01:27.739,2024-07-14 10:01:27.739,Portal Dye,misc,76500000,False,0,2024-07-14 04:01:27.739,True
671,/viewauction 375f23ad32bb4c708458d7db971021ab,2024-07-14 03:59:46.253,2024-07-28 03:59:46.253,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:46.253,True
647,/viewauction 685a33170ec94aea99fb10e836c1da40,2024-07-14 03:59:55.304,2024-07-28 03:59:55.304,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:55.304,True
695,/viewauction 6082db19dc4044febd1cae95678b5705,2024-07-14 03:59:36.800,2024-07-28 03:59:36.800,Portal Dye,misc,77900000,False,0,2024-07-14 03:59:36.800,True
1852,/viewauction 4629c31edc974339aff69d14675eaca2,2024-07-14 03:52:11.537,2024-07-14 09:52:11.537,Portal Dye,misc,77999000,False,0,2024-07-14 03:52:11.537,True
166,/viewauction 2dba915d46fd4a81ba15d06ebd4f63d3,2024-07-14 04:02:37.503,2024-07-16 04:02:37.503,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:37.503,True
148,/viewauction fef4a9abccd94495b5fff45d2250b914,2024-07-14 04:02:43.288,2024-07-16 04:02:43.288,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:43.288,True
129,/viewauction ed54a7201784408bbf3c5978e18e95ff,2024-07-14 04:02:49.881,2024-07-16 04:02:49.881,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:49.881,True
107,/viewauction 768f2955a9364daba1a3eeec6f08006e,2024-07-14 04:02:56.285,2024-07-16 04:02:56.285,Portal Dye,misc,78000000,False,0,2024-07-14 04:02:56.285,True


In [299]:
# Search Dictionary 
# USAGE
# Key = item_name
# Value = Maximum value of item
# search_dict = {
#   'Key': Value,
#   'Key': Value,
# }
search_dict = {
    'Portal Dye':78000000,
    'Molten Cloak': 900000,
    'Molten Belt': 900000		
    }

def find_item(item_name:str, specified_price:int) -> pd.DataFrame:
    filtered_df = df[df['item_name'] == item_name]
    filtered_df = filtered_df.sort_values(by=['starting_bid'])
    filtered_df = filtered_df.query(f'starting_bid < {specified_price} and bin == True and claimed == False')
    filtered_df.loc[:,'uuid'] = '/viewauction ' + filtered_df['uuid'].astype(str)
    if filtered_df.empty:
        return pd.DataFrame(columns=df.columns)
    return filtered_df

df4 = pd.DataFrame()

# Loop through Dictionary
for item_name,specified_price in search_dict.items():
    result = find_item(item_name,specified_price)
    df4 = pd.concat([df4,result], ignore_index=True)
df4

,uuid,start,end,item_name,category,starting_bid,claimed,highest_bid_amount,last_updated,bin
0,/viewauction 083648e3993449929033129aaab91baf,2024-07-14 05:36:47.302,2024-07-16 05:36:47.302,Portal Dye,misc,77900000,False,0,2024-07-14 05:36:47.302,True
1,/viewauction d32f559d6fc4477d93b335abe2a1f60f,2024-07-14 05:36:39.096,2024-07-16 05:36:39.096,Portal Dye,misc,77900000,False,0,2024-07-14 05:36:39.096,True
2,/viewauction eb64e2f924ec46e0a270dfc4e2af15d4,2024-07-14 05:34:14.112,2024-07-14 11:34:14.112,Molten Cloak,misc,800000,False,0,2024-07-14 05:34:14.112,True
3,/viewauction 99609ed3fc294b249f2b019484627178,2024-07-14 05:20:55.841,2024-07-14 11:20:55.841,Molten Cloak,misc,899000,False,0,2024-07-14 05:20:55.841,True
4,/viewauction 0420ff52a4404213b4c8336c11b04867,2024-07-14 05:37:37.126,2024-07-14 11:37:37.126,Molten Belt,misc,800000,False,0,2024-07-14 05:37:37.126,True
5,/viewauction b3f41776b04e491090296cc2324b6159,2024-07-14 05:33:01.115,2024-07-16 05:33:01.115,Molten Belt,misc,800000,False,0,2024-07-14 05:33:01.115,True
